In [1]:
import pandas as pd
import json
import folium
from IPython.display import display

# Pad naar de bestanden
user_profiles_path = '../Data/Loopdata/kleinebestanden/run_gebruikersprofielen.json'
activities_path = '../Data/Loopdata/kleinebestanden/routes_volledig.json'

# Laad de bestanden
with open(user_profiles_path, 'r') as file:
    user_profiles_content = [json.loads(line) for line in file]
    user_profiles_df = pd.DataFrame(user_profiles_content)

with open(activities_path, 'r') as file:
    activities_content = [json.loads(line) for line in file]
    activities_df = pd.DataFrame(activities_content)

# Functie om een gebruiker te selecteren en de locatie en routes te visualiseren
def visualize_user_and_routes(user_id, user_profiles_df, activities_df):
    # Zoek de gebruiker en de bijbehorende activiteiten
    user_data = user_profiles_df[user_profiles_df['userId'] == user_id].iloc[0]
    user_location = user_data['average_location']
    user_activities_ids = user_data['activity_ids']

    # Filter de activiteiten die bij de gebruiker horen
    user_activities = activities_df[activities_df['id'].isin(user_activities_ids)]

    # Maak een kaartobject
    map_obj = folium.Map(location=user_location, zoom_start=8)

    # Voeg een marker toe voor de locatie van de gebruiker
    folium.Marker(user_location, popup=f"User: {user_id}").add_to(map_obj)

    # Functie om een lijn te tekenen voor elke activiteit
    def draw_activity(map_obj, activity):
        if activity['latitude'] and activity['longitude']:
            # Voeg lijnen toe om de activiteit te visualiseren
            points = list(zip(activity['latitude'], activity['longitude']))
            folium.PolyLine(points, color='blue', weight=2.5, opacity=1).add_to(map_obj)

    # Teken de activiteiten van de gebruiker
    for _, activity in user_activities.iterrows():
        draw_activity(map_obj, activity)

    # Toon de kaart
    display(map_obj)

# Voorbeeld: Visualiseer de gegevens van een specifieke gebruiker
# sample_user_id = 9225240  # Neem de eerste gebruiker als voorbeeld
first_user_id = user_profiles_df['userId'].iloc[0]
visualize_user_and_routes(first_user_id, user_profiles_df, activities_df)
